In [ ]:
# Make sure all necessary imports are included
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random


In [ ]:
# implementing a simple policy network for tic tac toe that outputs a probability distribution over all moves

class SimplePolicyNetwork(nn.Module):
    def __init__(self, board_size, num_moves):
        """
        Initializes the Policy Network.
        :param board_size: Tuple of the board dimensions, e.g., (19, 19) for Go.
        :param num_moves: Total number of possible moves in the game.
        """
        super(SimplePolicyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc = nn.Linear(128 * board_size[0] * board_size[1], num_moves)

    def forward(self, x):
        """
        Forward pass of the network.
        :param x: Input tensor, the game state.
        :return: Probability distribution over all possible moves.
        """
        # Apply two convolutional layers with ReLU activations
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)

        # Output layer with a softmax to get probabilities
        x = self.fc(x)
        return F.softmax(x, dim=1)

# Example usage
board_size = (3, 3)  # For Tic Tac Toe
num_moves = board_size[0] * board_size[1]  # Assuming each cell is a possible move
model = SimplePolicyNetwork(board_size, num_moves)

# Example input: a single game state, with 1 channel, and 19x19 board size
# The input should be a 4D tensor: [batch_size, channels, height, width]
# Here, batch_size = 1, channels = 1 (just the board, could be more for different game states)
game_state = torch.randn(1, 1, board_size[0], board_size[1])
# Get the probability distribution over moves
probabilities = model(game_state)

print(probabilities)  # Each element corresponds to the probability of a move being the best next move


In [ ]:
# Solving tic-tac-toe using tree search
# This is a simple implementation of the minimax algorithm

# The game is represented as a 3x3 matrix
# 0 represents an empty cell
# 1 represents a cell with a cross
# 2 represents a cell with a circle

class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.turn = 1
        self.winner = 0

    def is_full(self):
        return np.all(self.board)

    def is_winner(self, player):
        for i in range(3):
            if np.all(self.board[i] == player) or np.all(self.board[:, i] == player):
                return True
        if np.all(self.board.diagonal() == player) or np.all(np.fliplr(self.board).diagonal() == player):
            return True
        return False

    def is_game_over(self):
        # Check if any player has won
        for player in [1, 2]:
            if self.is_winner(player):
                self.winner = player
                return True
        # If no winner and the board is full, it's a draw
        if self.is_full():
            self.winner = 0  # Indicate a draw
            return True
        # Game is not over
        return False


    def get_valid_moves(self):
        return np.argwhere(self.board == 0)
    
    def get_valid_moves_indices(self):
        return np.flatnonzero(self.board == 0)
    

    def make_move(self, move):
        self.board[tuple(move)] = self.turn
        self.turn = 3 - self.turn

    def make_move_from_index(self, index):
        move = np.unravel_index(index, (3, 3))
        self.make_move(move)

    def undo_move(self, move):
        self.board[tuple(move)] = 0
        self.turn = 3 - self.turn

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)

    def __str__(self):
        return str(self.board)


In [ ]:
# game = TicTacToe()
# game.make_move((1,1))
# print(game)

# game.make_move((2,1))
# print(game)

# game.make_move((0,1))
# print(game)

In [ ]:
def convert_board_to_input(board):
    """
    Convert the game board to a tensor suitable for the policy network.
    The input is a 4D tensor: [batch_size, channels, height, width].
    """
    # Convert the board to a tensor with shape (1, 1, 3, 3)
    # 1 channel, the board's state is represented in a 3x3 grid
    board_tensor = torch.tensor(board, dtype=torch.float).unsqueeze(0).unsqueeze(0)
    return board_tensor

def select_move(probabilities, valid_moves_indices):
    """
    Select the move with the highest probability that is also a valid move.
    """
    # Zero out the probabilities of moves that are not valid
    prob_masked = probabilities.clone().detach()
    prob_masked[0, np.setdiff1d(np.arange(num_moves), valid_moves_indices)] = 0
    # Select the move with the highest probability
    move_index = torch.argmax(prob_masked).item()
    return move_index

# Initialize the TicTacToe game
game = TicTacToe()

# Initialize the policy network
model = SimplePolicyNetwork(board_size, num_moves)

# Play until the game is over
while not game.is_game_over():
    # Convert the current game state to a tensor input for the network
    current_state_tensor = convert_board_to_input(game.board)
    # Get the probability distribution over moves from the policy network
    probabilities = model(current_state_tensor)
    # Get valid move indices
    valid_moves_indices = game.get_valid_moves_indices()
    # Select the move with the highest probability among valid moves
    selected_move_index = select_move(probabilities, valid_moves_indices)
    # Make the move
    game.make_move_from_index(selected_move_index)
    # Print the board state
    print(game)
    print("------")

# Check the result
if game.winner:
    print(f"Player {game.winner} wins!")
else:
    print("It's a draw!")

In [ ]:
import matplotlib.pyplot as plt

import random

def play_game_with_random(policy_model, game, random_agent, start_first=True):
    game.reset()
    # Randomly choose which agent starts
    game.turn = random.choice([1, 2])
    while not game.is_game_over():
        if game.turn == 1:  # Policy network's turn
            current_state_tensor = convert_board_to_input(game.board)
            probabilities = policy_model(current_state_tensor)
            valid_moves_indices = game.get_valid_moves_indices()
            selected_move_index = select_move(probabilities, valid_moves_indices)
            game.make_move_from_index(selected_move_index)
        else:  # Random agent's turn
            valid_moves_indices = game.get_valid_moves_indices()
            selected_move_index = random_agent.select_move(valid_moves_indices)
            game.make_move_from_index(selected_move_index)
    return game.winner

wins = {1: 0, 2: 0, 0: 0}  # 1: Policy Network, 2: Random Agent, 0: Draw
num_simulations = 5000

for _ in range(num_simulations):
    winner = play_game_with_random(model, game, random_agent, start_first)
    wins[winner] += 1
    game.reset()  # Reset the game to its initial state after each simulation


# Plotting win rates
labels = ['Policy Network Wins', 'Random Agent Wins', 'Draws']
values = [wins[1], wins[2], wins[0]]

plt.figure(figsize=(10, 6))
plt.bar(labels, values, color=['blue', 'red', 'green'])
plt.title('Win Rates After 100 Simulations')
plt.ylabel('Number of Games')
plt.show()


In [ ]:
# Defining a function to simulate a game between two random agents
def play_game_random_vs_random(game, agent1, agent2):
    game.reset()
    # Randomly choose which agent starts
    game.turn = random.choice([1, 2])
    while not game.is_game_over():
        if game.turn == 1:  # Random agent 1's turn
            valid_moves_indices = game.get_valid_moves_indices()
            selected_move_index = agent1.select_move(valid_moves_indices)
            game.make_move_from_index(selected_move_index)
        else:  # Random agent 2's turn
            valid_moves_indices = game.get_valid_moves_indices()
            selected_move_index = agent2.select_move(valid_moves_indices)
            game.make_move_from_index(selected_move_index)
    return game.winner

# Initializing the game and two random agents
game = TicTacToe()
random_agent1 = RandomAgent()
random_agent2 = RandomAgent()

gameHistory = []

# Simulating games
wins_random_vs_random = {1: 0, 2: 0, 0: 0}  # 1: Random Agent 1, 2: Random Agent 2, 0: Draw
num_simulations_random_vs_random = 5000

for _ in range(num_simulations_random_vs_random):
    winner = play_game_random_vs_random(game, random_agent1, random_agent2)
    wins_random_vs_random[winner] += 1
    # Store a copy of the board's state and the winner at this point
    final_state = (np.copy(game.board), game.winner)
    gameHistory.append(final_state)
    game.reset()  # Reset the game to its initial state after each simulation

# Plotting win rates for games between two random agents
labels_random_vs_random = ['Random Agent 1 Wins', 'Random Agent 2 Wins', 'Draws']
values_random_vs_random = [wins_random_vs_random[1], wins_random_vs_random[2], wins_random_vs_random[0]]

plt.figure(figsize=(10, 6))
plt.bar(labels_random_vs_random, values_random_vs_random, color=['blue', 'red', 'green'])
plt.title('Win Rates After 5000 Simulations: Random vs. Random')
plt.ylabel('Number of Games')
plt.show()



In [ ]:
import torch.optim as optim


In [ ]:
def compute_returns(rewards, gamma=1.0):
    """
    Compute returns for each time step, given the rewards
    and a discount factor gamma.
    """
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    return returns

# Initialize environment, policy network, and optimizer
game = TicTacToe()
policy_network = SimplePolicyNetwork(board_size=(3, 3), num_moves=9)
optimizer = optim.Adam(policy_network.parameters(), lr=0.001)

num_episodes = 10000


def get_reward(game):
    """
    Defines the reward for the policy network based on the game's outcome.
    Assume the policy network always plays as player 1.
    """
    if game.is_winner(1):  # Policy network wins
        return 1
    elif game.is_winner(2):  # Policy network loses
        return -1
    return 0  # No reward for intermediate moves


win_record = []
policy_losses = []  # Store policy losses for plotting
episode_rewards = []  # Average reward per episode

# Training loop
for episode in range(num_episodes):
    saved_log_probs = []
    rewards = []
    game.reset()
    while not game.is_game_over():
        if game.turn == 1:  # Policy network's turn
            state = convert_board_to_input(game.board)
            probs = policy_network(state)
            action = torch.multinomial(probs, 1).item()  # Sample action
            saved_log_probs.append(torch.log(probs.squeeze(0)[action]))
            game.make_move_from_index(action)
        else:  # Random agent's turn
            valid_moves = game.get_valid_moves_indices()
            action = np.random.choice(valid_moves)
            game.make_move_from_index(action)
        reward = get_reward(game)  # Define a suitable reward function
        rewards.append(reward)
    
    # Compute returns
    returns = compute_returns(rewards)
    returns = torch.tensor(returns)

    # Determine the outcome of the episode
    win_record.append(game.winner)
    episode_rewards.append(np.mean(rewards))
    
    # Collect individual loss tensors in a list
    policy_loss_terms = []
    for log_prob, R in zip(saved_log_probs, returns):
        loss = -log_prob * R
        policy_loss_terms.append(loss.unsqueeze(0))

    # Concatenate and sum to compute the total policy loss for the episode
    total_policy_loss = torch.cat(policy_loss_terms).sum()
    policy_losses.append(total_policy_loss.item())  # Recording scalar loss

    # Use total_policy_loss for gradient computation
    optimizer.zero_grad()
    total_policy_loss.backward()
    optimizer.step()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to calculate rolling average using np.convolve
def calculate_rolling_average(data, window_size=10):
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, 'valid')

# Convert win_record to binary for win, loss, and draw
wins_binary = (np.array(win_record) == 1).astype(float)
losses_binary = (np.array(win_record) == 2).astype(float)
draws_binary = (np.array(win_record) == 0).astype(float)

# Calculate rolling averages
rolling_avg_win_rate = calculate_rolling_average(wins_binary)
rolling_avg_loss_rate = calculate_rolling_average(losses_binary)
rolling_avg_draw_rate = calculate_rolling_average(draws_binary)
rolling_avg_policy_losses = calculate_rolling_average(policy_losses)
rolling_avg_episode_rewards = calculate_rolling_average(episode_rewards)

# Plotting
window_size = 10  # Adjust window_size if needed

plt.figure(figsize=(14, 8))

# Win, Loss, Draw Rates
plt.subplot(3, 1, 1)
episodes_adjusted = np.arange(window_size, len(win_record) + 1)
plt.plot(episodes_adjusted, rolling_avg_win_rate, label='Win Rate')
plt.plot(episodes_adjusted, rolling_avg_loss_rate, label='Loss Rate')
plt.plot(episodes_adjusted, rolling_avg_draw_rate, label='Draw Rate')
plt.xlabel('Episodes')
plt.ylabel('Rate')
plt.title('Performance Over Time (Rolling Avg)')
plt.legend()

# Policy Loss
plt.subplot(3, 1, 2)
plt.plot(episodes_adjusted, rolling_avg_policy_losses, label='Policy Loss', color='red')
plt.xlabel('Episodes')
plt.ylabel('Loss')
plt.title('Policy Loss Over Time (Rolling Avg)')
plt.legend()

# Average Rewards
plt.subplot(3, 1, 3)
plt.plot(episodes_adjusted, rolling_avg_episode_rewards, label='Average Reward', color='green')
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward Per Episode (Rolling Avg)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# self play

def compute_returns(rewards, gamma=1.0):
    """
    Compute returns for each time step, given the rewards
    and a discount factor gamma.
    """
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    return returns

# Initialize environment, policy network, and optimizer
game = TicTacToe()
policy_network = SimplePolicyNetwork(board_size=(3, 3), num_moves=9)
optimizer = optim.Adam(policy_network.parameters(), lr=0.001)

num_episodes = 10000


def get_reward(game):
    """
    Defines the reward for the policy network based on the game's outcome.
    Assume the policy network always plays as player 1.
    """
    if game.is_winner(1):  # Policy network wins
        return 1
    elif game.is_winner(2):  # Policy network loses
        return -1
    return 0  # No reward for intermediate moves


win_record = []
policy_losses = []  # Store policy losses for plotting
episode_rewards = []  # Average reward per episode

# Training loop
for episode in range(num_episodes):
    saved_log_probs = []
    rewards = []
    game.reset()
    while not game.is_game_over():
        if game.turn == 1:  # Policy network's turn
            state = convert_board_to_input(game.board)
            probs = policy_network(state)
            action = torch.multinomial(probs, 1).item()  # Sample action
            saved_log_probs.append(torch.log(probs.squeeze(0)[action]))
            game.make_move_from_index(action)
        else:  # Random agent's turn
            valid_moves = game.get_valid_moves_indices()
            action = np.random.choice(valid_moves)
            game.make_move_from_index(action)
        reward = get_reward(game)  # Define a suitable reward function
        rewards.append(reward)
    
    # Compute returns
    returns = compute_returns(rewards)
    returns = torch.tensor(returns)

    # Determine the outcome of the episode
    win_record.append(game.winner)
    episode_rewards.append(np.mean(rewards))
    
    # Collect individual loss tensors in a list
    policy_loss_terms = []
    for log_prob, R in zip(saved_log_probs, returns):
        loss = -log_prob * R
        policy_loss_terms.append(loss.unsqueeze(0))

    # Concatenate and sum to compute the total policy loss for the episode
    total_policy_loss = torch.cat(policy_loss_terms).sum()
    policy_losses.append(total_policy_loss.item())  # Recording scalar loss

    # Use total_policy_loss for gradient computation
    optimizer.zero_grad()
    total_policy_loss.backward()
    optimizer.step()

